# Researcher Chat with WatsonX
Hello, everyone. The purpose of this notebook is to create a chatbot that reads papers online and answers technical questions from  Arxiv.

You can download one of the best models of Generative AI that are  free and open source or use the more advanced ones within WatsonX. 

The idea of this program is to develop a simple tool that helps researchers find insights about issues that we have in science.



### First we need to load our libraries needed for this task

In [1]:
import urllib.request
import fitz
import re
import numpy as np
import tensorflow_hub as hub
import gradio as gr
import os
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
# Compability with Hugging Face models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
# Compability with ChatGPT
import openai

In [2]:
DEFAULT_FILE_URL = 'https://arxiv.org/pdf/1706.03762.pdf'
DEFAULT_FILE = '1706.03762.pdf'
GPT_3_5_TURBO = 'gpt-3.5-turbo'
GPT_3_5_TURBO_16K = 'gpt-3.5-turbo-16k'
GPT_3_5_TURBO_0613 = 'gpt-3.5-turbo-0613'
GPT_3_5_TURBO_16K_0613 = 'gpt-3.5-turbo-16k-0613'
GPT_4 = 'gpt-4'
GPT_4_32K = 'gpt-4-32k'
TEXT_DAVINCI_003 = 'text-davinci-003'

In [3]:
try:
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
except KeyError:
    OPENAI_API_KEY = input("Please enter your api key (hit enter): ")

In [4]:
#OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
#AUTH_USER = os.environ.get("AUTH_USER")
#AUTH_PASSWORD = os.environ.get("AUTH_PASSWORD")

In [5]:
if len(OPENAI_API_KEY) !=51 :
    OPENAI_API_KEY = input("Please enter your api key (hit enter): ")

In [6]:
BASE_PROMPT_TEXT_DAVINCI_003 = ""\
    "Compose a comprehensive reply to the query using the search results given. "\
    "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
    "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
    "with the same name, create separate answers for each. Only include information found in the results and "\
    "don't add any additional information. Make sure the answer is correct and don't output false content. "\
    "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
    "search results which has nothing to do with the question. Only answer what is asked. The "\
    "answer should be short and concise."

BASE_PROMPT_GPT = ""\
    "Instructions: Compose a comprehensive reply to the Query using the SearchResults given. "\
    "Cite each reference using [ Page Number] notation. "\
    "Only answer what is asked. The answer should be short and concise."

PROMPT_DAVINCI = BASE_PROMPT_TEXT_DAVINCI_003
PROMPT_GPT = BASE_PROMPT_GPT 

In [7]:
def reset():
    globals().update(PROMPT_DAVINCI=BASE_PROMPT_TEXT_DAVINCI_003)
    globals().update(PROMPT_GPT=BASE_PROMPT_GPT)
    return 'Reset Successful.'

In [8]:
def download_pdf(url, output_path):
    urllib.request.urlretrieve(url, output_path)

In [9]:
def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text

In [10]:
def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        print(f'Log: Page {i+1}/{total_pages} loaded with text: {text}')
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list

In [11]:
def text_to_chunks(texts, word_length=150, start_page=1):
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []
    
    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[Page no. {idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [12]:
class SemanticSearch:
    
    def __init__(self):
        #self.use = hub.load('./universal-sentence-encoder_4/')
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False
    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True
    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]
        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors
    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [25]:
def load_recommender(path, start_page=1):
    global recommender
    texts = pdf_to_text(path, start_page=start_page)
    chunks = text_to_chunks(texts, start_page=start_page)
    recommender.fit(chunks)
    return 'Corpus Loaded.'

In [26]:
def generate_text_gpt(openAI_key, prompt, model):
    openai.api_key = openAI_key
    temperature=0.7
    max_tokens=256
    top_p=1
    frequency_penalty=0
    presence_penalty=0

    message = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": PROMPT_GPT},
            {"role": "user", "content": "What is the maximum capacity?"},
            {"role": "assistant", "content": "The maximum AC continuous output power is 29.99 kW [Page no. 13]."},
            {"role": "user", "content": prompt}
        ],
        temperature=.3,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    ).choices[0].message['content']
    return message

In [27]:
def make_search_results(question):
    topn_chunks = recommender(question)
    search_results = ''

    for c in topn_chunks:
        search_results += c + '\n\n'
    return search_results

In [28]:
def set_base_prompt_GPT(prompt):
    global PROMPT_GPT
    PROMPT_GPT = prompt
    return "OK"

In [29]:
def base_prompt_GPT():
    return BASE_PROMPT_TEXT_DAVINCI_003
  
def generate_answer_GPT(question, openAI_key, model):
    search_results = make_search_results(question)
    
    prompt = ""
    prompt += f"SearchResults:\n\n{search_results}"
    prompt += f"Query: {question}"

    answer = generate_text_gpt(openAI_key, prompt, model)
    return answer, PROMPT_GPT, prompt

In [30]:
def question_answer(url, file, question, model):
    try:
        openAI_key = OPENAI_API_KEY
        if openAI_key.strip()=='':
            return '[ERROR]: Failed to load OpenAI API Key from environment variable.'
        if model is None or model =='':
            return '[ERROR]: No model selected.'
        if url.strip() != '':
            glob_url = url
            download_pdf(glob_url, 'corpus.pdf')
            load_recommender('corpus.pdf')
        if file is not None and file.name != '':
            old_file_name = file.name
            file_name = file.name
            file_name = file_name[:-12] + file_name[-4:]
            os.rename(old_file_name, file_name)
            load_recommender(file_name)
        else:
            load_recommender(Path(DEFAULT_FILE))
        
        if question.strip() == '':
            return '[ERROR]: Question field is empty'
        if model == TEXT_DAVINCI_003:
            return generate_answer_davinci_003(question, openAI_key)
        
        return generate_answer_GPT(question, openAI_key, model)
    except openai.error.InvalidRequestError as e:
        return f'[ERROR]: Either you do not have access to GPT4 or you have exhausted your quota!'


In [31]:
def generate_text_davinci(openAI_key, prompt):
    try:
        openai.api_key = openAI_key
        completions = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=512,
            n=1,
            stop=None,
            temperature=0.7,
        )
        message = completions.choices[0].text
    except Exception as e:
        message = f'API Error: {str(e)}'
    return message 


In [32]:
def set_base_prompt_davinci_003(prompt):
    global PROMPT_DAVINCI
    PROMPT_DAVINCI = prompt
    return "OK"

In [33]:
def base_prompt_davinci_003():
    return BASE_PROMPT_TEXT_DAVINCI_003

In [34]:
def generate_answer_davinci_003(question,openAI_key):
    search_results = make_search_results(question)

    prompt = ""
    prompt += f"Search Results:\n\n{search_results}"
    prompt += f"Instructions: {BASE_PROMPT_TEXT_DAVINCI_003}\n\n"
    prompt += "Query: {question}\n\nAnswer:{answer}\n\n"
    prompt += f"Query: {question}\n\n"
    prompt += "Answer:"

    answer = generate_text_davinci(openAI_key, prompt)
    return answer, PROMPT_DAVINCI, prompt

In [35]:
# pre-defined questions
questions = [
    "What did the study investigate?",
    "Can you provide a summary of this paper?",
    "what are the methodologies used in this study?",
    "what are the data intervals used in this study? Give me the start dates and end dates?",
    "what are the main limitations of this study?",
    "what are the main shortcomings of this study?",
    "what are the main findings of the study?",
    "what are the main results of the study?",
    "what are the main contributions of this study?",
    "what is the conclusion of this paper?",
    "what are the input features used in this study?",
    "what is the dependent variable in this study?",
]

In [37]:
recommender = SemanticSearch()
title = 'Chat with PDF'
with gr.Blocks(css="""#response { font-size: 14px; min-height: 1200; }""") as demo:
    gr.Markdown(f'<center><h3>{title}</h3></center>')
    with gr.Row():
        with gr.Group():
            question = gr.Textbox(label='Enter your question here')           
            btn = gr.Button(value='Submit')
            btn.style(full_width=True)
            with gr.Accordion(label="Select PDF", open=False):
                url = gr.Textbox(label=f'Enter PDF URL here  (Example: {DEFAULT_FILE_URL})')
                gr.Markdown("<center><h7>OR</h7></center>")
                file = gr.File(label='Upload your PDF/ Research Paper / Book here', file_types=['.pdf'])
            
            model = gr.Radio([
                'gpt-3.5-turbo', 
                'gpt-3.5-turbo-16k', 
                'gpt-3.5-turbo-0613', 
                'gpt-3.5-turbo-16k-0613', 
                'text-davinci-003',
                'gpt-4',
                'gpt-4-32k'
            ], label='Select Model', value='gpt-3.5-turbo')
            with gr.Accordion(label="Prompt", open=False):
                gpt_prompt = gr.Textbox(label=f'GPT model prompt')
                btn_gpt_prompt = gr.Button(value='Submit GPT Prompt')
                btn_gpt_prompt.style(full_width=True)
                gr.Markdown("<center><h7>OR</h7></center>")
                davinci_prompt = gr.Textbox(label=f'Davinci model prompt')
                btn_davinci_prompt = gr.Button(value='Submit Davinci Prompt')
                btn_davinci_prompt.style(full_width=True)
            
            btn_reset = gr.Button(value='Reset')

        with gr.Group():
            response = gr.Label(label="Response", elem_id="response")
            with gr.Accordion(label="Log", open=False):
                system = gr.Textbox(label="System")
                prompt = gr.Textbox(label="Prompt")
    btn.click(
        question_answer,
        inputs=[url, file, question, model],
        outputs=[response,system,prompt],
        api_name="question_answer"
    )
    btn_gpt_prompt.click(set_base_prompt_GPT, inputs=[gpt_prompt], outputs=[], api_name="set_base_prompt_GPT")
    btn_davinci_prompt.click(set_base_prompt_davinci_003, inputs=[davinci_prompt], outputs=[], api_name="set_base_prompt_davinci_003")
    btn_reset.click(reset, inputs=[], outputs=[], api_name="reset")

In [38]:
demo.launch(show_api=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [39]:
# Compability with Hugging Face models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch

In [50]:
def query(query,modelo="flan-t5-small",tokens=100000):
    options=""
    tok_len=tokens
    t5query = f"""Question: "{query}" Context: {options}"""        
    if (modelo=="flan-t5-small" or modelo=="flan-t5-large"):
       tokenizer = AutoTokenizer.from_pretrained("google/{}".format(modelo))
       model = AutoModelForSeq2SeqLM.from_pretrained("google/{}".format(modelo)).to(device)
       inputs = tokenizer(t5query, return_tensors="pt").to(device)
       outputs = model.generate(**inputs, max_new_tokens=tok_len)
    else:
        model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").to(device)
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
        input_ids = tokenizer(t5query, return_tensors="pt").to(device) 
        outputs = model.generate(**input_ids, do_sample=True, max_length=tok_len) 
    generation=tokenizer.batch_decode(outputs, skip_special_tokens=True)    
    return '\n'.join(generation)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
# Testing the enviroment
question='What is the capital of Italy'
query(question, modelo ="flan-t5-small" ,tokens=100)

'rome'

In [43]:
recommender = SemanticSearch()

In [53]:
def generate_answer(question,openAI_key=None):
    topn_chunks = recommender(question)
    prompt = ""
    prompt += 'search results:\n\n'
    for c in topn_chunks:
        prompt += c + '\n\n'
    prompt += "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise. \n\nQuery: {question}\nAnswer: "
    
    prompt += f"Query: {question}\nAnswer:"
    answer = prompt
    #answer = generate_text(openAI_key, prompt,"text-davinci-003")
    return answer

In [13]:
generate_answer(question)/

SyntaxError: invalid syntax (2036064751.py, line 1)

In [48]:
def question_answer(url, file, question,openAI_key):
    if openAI_key.strip()=='':
        return '[ERROR]: Please enter you Open AI Key. Get your key here : https://platform.openai.com/account/api-keys'
    if url.strip() == '' and file == None:
        return '[ERROR]: Both URL and PDF is empty. Provide atleast one.'
    
    if url.strip() != '' and file != None:
        return '[ERROR]: Both URL and PDF is provided. Please provide only one (eiter URL or PDF).'
    if url.strip() != '':
        glob_url = url
        download_pdf(glob_url, 'corpus.pdf')
        load_recommender('corpus.pdf')
    else:
        old_file_name = file.name
        file_name = file.name
        file_name = file_name[:-12] + file_name[-4:]
        os.rename(old_file_name, file_name)
        load_recommender(file_name)

    if question.strip() == '':
        return '[ERROR]: Question field is empty'
    return generate_answer(question,openAI_key)

In [49]:
#def run_openai():
recommender = SemanticSearch()
title = 'PDF GPT'
description = """ PDF GPT allows you to chat with your PDF file using Universal Sentence Encoder and Open AI. It gives hallucination free response than other tools as the embeddings are better than OpenAI. The returned response can even cite the page number in square brackets([]) where the information is located, adding credibility to the responses and helping to locate pertinent information quickly."""   
with gr.Blocks() as demo:
    gr.Markdown(f'<center><h1>{title}</h1></center>')
    gr.Markdown(description)
    with gr.Row():
        with gr.Group():
            gr.Markdown(f'<p style="text-align:center">Get your Open AI API key <a href="https://platform.openai.com/account/api-keys">here</a></p>')
            openAI_key=gr.Textbox(label='Enter your OpenAI API key here')
            url = gr.Textbox(label='Enter PDF URL here')
            gr.Markdown("<center><h4>OR<h4></center>")
            file = gr.File(label='Upload your PDF/ Research Paper / Book here', file_types=['.pdf'])
            question = gr.Textbox(label='Enter your question here')
            btn = gr.Button(value='Submit')
            btn.style(full_width=True)
        with gr.Group():
            answer = gr.Textbox(label='The answer to your question is :')
        btn.click(question_answer, inputs=[url, file, question,openAI_key], outputs=[answer])
demo.launch()
    

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [37]:
#run_openai()

In [ ]:
def generate_text_text_davinci_003(openAI_key,prompt, engine="text-davinci-003"):
    openai.api_key = openAI_key
    completions = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.7,
    )
    message = completions.choices[0].text
    return message

# pre-defined questions
questions = [
    "What did the study investigate?",
    "Can you provide a summary of this paper?",
    "what are the methodologies used in this study?",
    "what are the data intervals used in this study? Give me the start dates and end dates?",
    "what are the main limitations of this study?",
    "what are the main shortcomings of this study?",
    "what are the main findings of the study?",
    "what are the main results of the study?",
    "what are the main contributions of this study?",
    "what is the conclusion of this paper?",
    "what are the input features used in this study?",
    "what is the dependent variable in this study?",
]

In [14]:
def run_turbo():
    title = 'PDF GPT Turbo'
    description = """ PDF GPT Turbo allows you to chat with your PDF files. It uses Google's Universal Sentence Encoder with Deep averaging network (DAN) to give hallucination free response by improving the embedding quality of OpenAI. It cites the page number in square brackets([Page No.]) and shows where the information is located, adding credibility to the responses."""
    with gr.Blocks(css="""#chatbot { font-size: 14px; min-height: 1200; }""") as demo:
    
        gr.Markdown(f'<center><h3>{title}</h3></center>')
        gr.Markdown(description)
    
        with gr.Row():
            
            with gr.Group():
                gr.Markdown(f'<p style="text-align:center">Get your Open AI API key <a href="https://platform.openai.com/account/api-keys">here</a></p>')
                with gr.Accordion("API Key"):
                    openAI_key = gr.Textbox(label='Enter your OpenAI API key here', password=True)
                    url = gr.Textbox(label='Enter PDF URL here   (Example: https://arxiv.org/pdf/1706.03762.pdf )')
                    gr.Markdown("<center><h4>OR<h4></center>")
                    file = gr.File(label='Upload your PDF/ Research Paper / Book here', file_types=['.pdf'])
                question = gr.Textbox(label='Enter your question here')
                gr.Examples(
                    [[q] for q in questions],
                    inputs=[question],
                    label="PRE-DEFINED QUESTIONS: Click on a question to auto-fill the input box, then press Enter!",
                )
                model = gr.Radio([
                    'gpt-3.5-turbo', 
                    'gpt-3.5-turbo-16k', 
                    'gpt-3.5-turbo-0613', 
                    'gpt-3.5-turbo-16k-0613', 
                    'text-davinci-003',
                    'gpt-4',
                    'gpt-4-32k'
                ], label='Select Model', default='gpt-3.5-turbo')
                btn = gr.Button(value='Submit')
                btn.style(full_width=True)
            with gr.Group():
                chatbot = gr.Chatbot(placeholder="Chat History", label="Chat History", lines=50, elem_id="chatbot")
        # Bind the click event of the button to the question_answer function
        btn.click(
            question_answer,
            inputs=[chatbot, url, file, question, openAI_key, model],
            outputs=[chatbot],
        )
    
    demo.launch()

In [18]:
run_turbo()

C:\Users\066226758\AppData\Local\Temp\ipykernel_26736\1997188045.py:14: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'password': True}
  openAI_key = gr.Textbox(label='Enter your OpenAI API key here', password=True)
C:\Users\066226758\AppData\Local\Temp\ipykernel_26736\1997188045.py:24: UserWarning: You have unused kwarg parameters in Radio, please remove them: {'default': 'gpt-3.5-turbo'}
  model = gr.Radio([
C:\Users\066226758\AppData\Local\Temp\ipykernel_26736\1997188045.py:36: UserWarning: You have unused kwarg parameters in Chatbot, please remove them: {'placeholder': 'Chat History', 'lines': 50}
  chatbot = gr.Chatbot(placeholder="Chat History", label="Chat History", lines=50, elem_id="chatbot")
C:\ArxivChat\.arxiv_environment\lib\site-packages\gradio\utils.py:758: UserWarning: Expected 4 arguments for function <function question_answer at 0x0000019F8E249120>, received 6.
  warnings.warn(
C:\ArxivChat\.arxiv_environment\lib\site-packages\gradio\u

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\fastapi\applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\starlette\applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "C:\ArxivChat\.arxiv_environment\lib\site-packages\starlette\middleware\errors.py", line 162, in __call__
    await self.app(scope, 

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [23]:
def generate_prompt(question):
    
    topn_chunks = recommender(question)
    results = ""
    results += 'search results:\n\n'
    for c in topn_chunks:
        results += c + '. \n\n'       
    instruction = "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise."
    
    prompt = instruction + "\n\nQuery: {}".format(question) + " \n\n" + results + " \nAnswer:"
    
    #answer = generate_text(openAI_key, prompt,"text-davinci-003")
    #return answer
    return prompt


In [24]:
question="Give me a summary of the abstract"
generate_prompt(question)

AttributeError: 'SemanticSearch' object has no attribute 'nn'

In [69]:
import arxiv
search = arxiv.Search(
  query = "Shell model",
  max_results = 2,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
  print(result.title)

Learning Disentangled Avatars with Hybrid 3D Representations
Attention De-sparsification Matters: Inducing Diversity in Digital Pathology Representation Learning


In [85]:
titles = [result.title for result in search.results()]
authors = [result.authors for result in search.results()]
summary = [result.summary for result in search.results()]
entry_id = [result.entry_id for result in search.results()]
#download_source = [result.download_source for result in search.results()]
#download_pdf = [result.download_pdf for result in search.results()]
pdf_url = [result.pdf_url for result in search.results()]
categories = [result.categories for result in search.results()]
comment = [result.comment for result in search.results()]
doi = [result.doi for result in search.results()]
published = [result.published for result in search.results()]

In [71]:
import pandas as pd

In [90]:
df = pd.DataFrame({
    'title': titles,
    'authors': authors,
    'summary': summary,
    #'entry_id': entry_id,
    'pdf_url':pdf_url,
    'categories':categories,
    'published':published
    
})

In [91]:
df.insert(0, 'id', range(1, len(df) + 1))

In [92]:
print(f'{df.shape[0]} results were found.')

2 results were found.


In [93]:
df

,id,title,authors,summary,pdf_url,categories,published
0,1,Learning Disentangled Avatars with Hybrid 3D R...,"[Yao Feng, Weiyang Liu, Timo Bolkart, Jinlong ...",Tremendous efforts have been made to learn ani...,http://arxiv.org/pdf/2309.06441v1,"[cs.CV, cs.AI, cs.GR]",2023-09-12 17:59:36+00:00
1,2,Attention De-sparsification Matters: Inducing ...,"[Saarthak Kapse, Srijan Das, Jingwei Zhang, Ra...","We propose DiRL, a Diversity-inducing Represen...",http://arxiv.org/pdf/2309.06439v1,[cs.CV],2023-09-12 17:59:10+00:00


In [94]:
# python program to check if a path exists
#if path doesn’t exist we create a new path
from pathlib import Path
#creating a new directory called pythondirectory
Path("./papers").mkdir(parents=True, exist_ok=True)

In [77]:
# Method 2 to download pdfs
id=1
for result in search.results():
    if len(df)!=0:
        result.download_pdf(filename="./papers/downloaded-paper-{}.pdf".format(id))
        id=id+1
    else:
        print('No results were found.')

In [117]:
# Method 2 to download pdfs
from pathlib import Path
import requests
id=1
for link in pdf_url:
    print(link)
    if len(pdf_url)!=0:
        filename_="./papers/downloaded-paper-{}.pdf".format(id)
        filename = Path(filename_)
        url = link
        response = requests.get(url)
        filename.write_bytes(response.content)
        id=id+1
    else:
        print('No results were found.')

http://arxiv.org/pdf/2309.06441v1
http://arxiv.org/pdf/2309.06439v1


In [118]:
# Method 3 to download pdfs
import sys
import pyxiv
for arxiv_id in entry_id:
    arxiv_id=arxiv_id.replace("http://arxiv.org/abs/","")
    print(arxiv_id)
    if len(entry_id)!=0:
        eprint = pyxiv.eprint_from_arxiv_id(arxiv_id)
        eprint.download("./papers")
    else:
        print('No results were found.')

2309.06441v1
2309.06439v1


In [ ]:
pyxiv.ePrint.download?

In [147]:
#dir(eprint)

In [ ]:
Notes
https://pypi.org/project/scidownl/
http://lukasschwab.me/arxiv.py/index.html#example-fetching-results-with-a-custom-client
https://github.com/blairbilodeau/arxiv-biorxiv-search/blob/master/arxiv_search_walkthrough.ipynb
https://github.com/Tachyu/Arxiv-download
https://github.com/tom2rd/arXivSearchPython
https://github.com/enriquecallejascastro/arXiv-scrape/blob/main/arXiv-scrape.ipynb
https://github.com/enriquecallejascastro/arXiv-scrape/blob/main/arXiv-scrape.ipynb
https://github.com/MorenoLaQuatra/arxiv_search
https://huggingface.co/spaces/Deci/DeciLM-6b-instruct

In [149]:
#paper = next(arxiv.Search(id_list=["1605.08386v1"]).results())
# Download the PDF to the PWD with a default filename.
#paper.download_pdf()

In [150]:
# Download the PDF to the PWD with a default filename.
#result.download_pdf()